In [2]:
import dropbox
import os

ACCESS_TOKEN = "sl.u.AFxWFrXGgvKl4lvpIN0KLrtJ4ncod7a5sDyXjSR3y4OG_RYmH5QZrjj0QGSay6cuWkGnjEXEskTwTx7yGE-QJQ2mPhxlXI_2Ej-ngSLYM0XGpJ1F7UUs_WqcHAgXPNt1r1W_jYBZHPiaPdMxLyPe8Wjs4-i9L8xWzSONS7zYi9yKDBM-yBkr9BiTLUViHoycPcmpOPQ6inY5OjQ-D7BhAmVl-WlN4gN7XXCnbGB3uC4scr5XbwNKSuQZN1jl9oFgzJdip4uyTcsZUYxc1Tk1GSvi1oLNUiSfVBrhWNhtA4ch5DwjkZoaQMc6a2lbRv8FIbiD91XYGvuCoEYNo9aKjcpjIqFVpLz8X9VXT894jcfjH2OwGFOR8tBBme9bPapFXVadBwvL996Frmn1-USL3tsIpU5NOzv8UlT2ez03bEzjGmasLf6ymdTnH4gg8cfZwiY2-SWEq2-cV5cgxDV1CaqMEucXUyodYlYM6ThqVktG71S-2xc1k76AIu6AfzW7lrUmxcrQSSqWAoy0DAPEsKfAmeqYi0ahou736kxNdy8ye3J3VfcM2Kvr_X1N7WNQLk8G8YF6iijWQQPTQK8iFsvoYXfXgmj29LIzcNnYGVIPUsv-YVfhMmFcLZc-Viwm--oujSlJDOTWtUsXvjU0QqP1RpW-ZfgaVOBZjg9DCa-IJMw-m4PvJ61Aq3VPLsjA1S5zOJ1A20U1VL-d8k0fTvQDcHAqDtIVSSa4JUdp2NfsCtY5oNWc40IYq3O2BKl65j9uNOFty7DlrGqKbvFzs6qQBgjUVPYNAM38azyG7n5rW2PvYxXJLGsFvZsCz5IX0Pa-b9b4T_hTGzcjMW2XRezgO9FiNF9tFMvgeFN3aPUcoeYtcgHLioFNs1CHZ7lA4BBZ8yZ3FeDx7WTv8dD27bfKsENLJget8B4jpxE5X-q9qtPdqp-BFkZKnvcdmmxU5OG4y7_xgx4zAlLt2Y_BvkojeUh7AKjmZ2Ai2H71duj97_JisVpVAEt2CFOeVvtNopZ6GbeSRHRnEreIzdH03nKkvncpefKQck7T2mCneZRp3cIf8kbGAm3u0PXlNIqfBYYcM3YOBTjAYdmBaTTFM9EsAN2VCC8gGPIu-1cE02uL-7NgaWdbMGspOXkbVHTTIiiQ8UBK5nbEzKo1yDKPfF3vCPruNB4RlHxrqBQPf679Mn4vJgU_0gZsHHv44Rn29UHFm9ohoLQCSsbSrZp7Tde8FP_SVtSliuC26iur6_t4jritLVR6DgbawW1Td_rZU507pnXLeZja3m0YH_ARMBHr3_Ey7WghULzougK8Vuw6XJQ8ylQGzF3Q8jexfQghcvIeEQZdGglN3yT_6c3pVRdCwiUIgwWwE6MvAYbzv-FrHg"
local_folder_path = "/Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples"
dropbox_base_folder_path = "/MIT/Jinyeop Song/MethlyGPT/data_examples" # You might want to adjust the Dropbox folder name

CHUNK_SIZE = 4 * 1024 * 1024  # 4MB

dbx = dropbox.Dropbox(ACCESS_TOKEN)

for root, dirs, files in os.walk(local_folder_path):
    for filename in files:
        local_file_path = os.path.join(root, filename)
        
        # Construct the relative path for Dropbox
        relative_path = os.path.relpath(local_file_path, local_folder_path)
        dropbox_file_path = os.path.join(dropbox_base_folder_path, relative_path)
        # On Windows, os.path.join uses backslashes, but Dropbox expects forward slashes
        dropbox_file_path = dropbox_file_path.replace("\\", "/")


        print(f"Uploading {local_file_path} to {dropbox_file_path}...")

        with open(local_file_path, "rb") as f:
            file_size = os.path.getsize(local_file_path) # More direct way to get file size

            if file_size <= CHUNK_SIZE:
                dbx.files_upload(f.read(), dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))
            else:
                upload_session_start_result = dbx.files_upload_session_start(f.read(CHUNK_SIZE))
                cursor = dropbox.files.UploadSessionCursor(
                    session_id=upload_session_start_result.session_id,
                    offset=f.tell()
                )
                commit = dropbox.files.CommitInfo(path=dropbox_file_path, mode=dropbox.files.WriteMode("overwrite"))

                while f.tell() < file_size:
                    if (file_size - f.tell()) <= CHUNK_SIZE:
                        dbx.files_upload_session_finish(f.read(CHUNK_SIZE), cursor, commit)
                    else:
                        dbx.files_upload_session_append_v2(f.read(CHUNK_SIZE), cursor)
                        cursor.offset = f.tell()
        print(f"Finished uploading {local_file_path}.")

print("All files uploaded.")


Uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE80969.csv.gz to /MIT/Jinyeop Song/MethlyGPT/data_examples/GSE80969.csv.gz...
Finished uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE80969.csv.gz.
Uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE225313.csv.gz to /MIT/Jinyeop Song/MethlyGPT/data_examples/GSE225313.csv.gz...
Finished uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE80969.csv.gz.
Uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE225313.csv.gz to /MIT/Jinyeop Song/MethlyGPT/data_examples/GSE225313.csv.gz...
Finished uploading /Entropy/Projects/multimodal/kying/A.Y/project/MethylGPT/tutorials/pretraining/data_examples/GSE225313.csv.gz.
Uploading /Entropy/Projects/multimodal/kying/A.Y/project/Me